In [ ]:
import numpy as np 
import pandas as pd 
import cellex
import csv

In [ ]:
## input data
data = pd.read_csv('your/pathway/CELLECT/Human_brain_cell_atlas/Spinal_cord_gene.csv', index_col=0)
data = data.T
data_32 = data.astype(np.float32)
metadata = pd.read_csv("your/pathway/CELLECT/metadata_caudate_scRNA.csv", index_col=0)


In [8]:
print(metadata.groupby('cell_type').cell_type.count())

cell_type
astrocyte                                   753
central nervous system macrophage           181
endothelial cell                              8
ependymal cell                              113
fibroblast                                   16
leukocyte                                     2
neuron                                    29889
oligodendrocyte                            1297
oligodendrocyte precursor cell              406
pericyte                                      9
vascular associated smooth muscle cell        4
Name: cell_type, dtype: int64


In [16]:
eso = cellex.ESObject(data=data_32, annotation=metadata, verbose=True)

Preprocessing - checking input ... input parsed in 0 min 0 sec
Preprocessing - running remove_non_expressed ... excluded 29 / 14476 genes in 0 min 2 sec
Preprocessing - normalizing data ... data normalized in 0 min 4 sec
Preprocessing - running ANOVA ... excluded 1090 / 14447 genes in 0 min 11 sec


In [17]:
eso.compute(verbose=True)

Computing DET ... 
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 4 sec
Computing EP ...
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 0 sec
Computing GES ...
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 4 sec
Computing NSI ...
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 0 sec
Computing ESmu ...
    finished in 0 min 0 sec
Computing ESsd ...
    finished in 0 min 0 sec
Computed ['det.esw', 'det.esw_null', 'det.pvals', 'det.esw_s', 'ep.esw', 'ep.esw_null', 'ep.pvals', 'ep.esw_s', 'ges.esw', 'ges.esw_null', 'ges.pvals', 'ges.esw_s', 'nsi.esw', 'nsi.esw_null', 'nsi.pvals', 'nsi.esw_s', 'esmu', 'essd'].


In [18]:
eso.results["esmu"].head()

,astrocyte,central nervous system macrophage,endothelial cell,ependymal cell,fibroblast,leukocyte,neuron,oligodendrocyte,oligodendrocyte precursor cell,pericyte,vascular associated smooth muscle cell
gene,,,,,,,,,,,
ENSG00000112182,0.075865,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.414634,0.368417,0.000000,0.000000
ENSG00000143499,0.000000,0.0,0.738272,0.403377,0.0,0.0,0.094473,0.000000,0.000000,0.000000,0.000000
ENSG00000139292,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.910498,0.612493,0.000000,0.000000
ENSG00000139304,0.000000,0.0,0.000000,0.760329,0.0,0.0,0.285511,0.000000,0.000000,0.000000,0.052832
ENSG00000101327,0.000000,0.0,0.171384,0.000000,0.0,0.0,0.260382,0.000000,0.000000,0.189547,0.000000


In [24]:
cellex.utils.mapping.human_ens_to_human_symbol(eso.results["esmu"], drop_unmapped=True, verbose=True)
cellex.utils.mapping.human_ens_to_human_symbol(eso.results["essd"], drop_unmapped=True, verbose=True)
eso.results["esmu"].head()

Mapping: human ensembl gene id's --> human gene symbols ...
Dataframe index contains values that are not ensemble format or not human ensembl id:  []
0.00 pct of genes are unmapped ...
Removed 0 unmapped genes ...
Mapping: human ensembl gene id's --> human gene symbols ...
0.00 pct of genes are unmapped ...
Removed 0 unmapped genes ...


,astrocyte,central nervous system macrophage,endothelial cell,ependymal cell,fibroblast,leukocyte,neuron,oligodendrocyte,oligodendrocyte precursor cell,pericyte,vascular associated smooth muscle cell
gene,,,,,,,,,,,
BACH2,0.075865,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.414634,0.368417,0.000000,0.000000
SMYD2,0.000000,0.0,0.738272,0.403377,0.0,0.0,0.094473,0.000000,0.000000,0.000000,0.000000
LGR5,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.910498,0.612493,0.000000,0.000000
PTPRQ,0.000000,0.0,0.000000,0.760329,0.0,0.0,0.285511,0.000000,0.000000,0.000000,0.052832
PDYN,0.000000,0.0,0.171384,0.000000,0.0,0.0,0.260382,0.000000,0.000000,0.189547,0.000000


In [25]:
## save
dirOut = "your/pathway/esmu"                    
prefixData = "Human_brain_cell_atlas_caudate" 
eso.save_as_csv(file_prefix=prefixData, path=dirOut, verbose=True)

Saving results as csv to disk ...
  Saved: /media/dai/devC/.jmm_project/CELLECT/STEAP/STEAP/esmu/Human_brain_cell_atlas_Bodycaudate_CaB.esmu.csv.gz
  Saved: /media/dai/devC/.jmm_project/CELLECT/STEAP/STEAP/esmu/Human_brain_cell_atlas_Bodycaudate_CaB.essd.csv.gz
Finished saving results to /media/dai/devC/.jmm_project/CELLECT/STEAP/STEAP/esmu
